In [ ]:
import argparse
import cv2
import numpy as np
import torch
from torchvision import models
import os
import glob
from pytorch_grad_cam import GradCAM, \
    HiResCAM, \
    ScoreCAM, \
    GradCAMPlusPlus, \
    AblationCAM, \
    XGradCAM, \
    EigenCAM, \
    EigenGradCAM, \
    LayerCAM, \
    FullGrad, \
    GradCAMElementWise
    
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
from pytorch_grad_cam import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    deprocess_image, \
    preprocess_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

def show(path):
      import matplotlib.pyplot as plt
      image=cv2.imread(path)
      original_width, original_height = image.shape[1], image.shape[0]
      resized_image = cv2.resize(image, (int(original_width/4), int(original_height/4)), interpolation = cv2.INTER_CUBIC)
      resized_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      plt.figure(figsize=(6,3))
      plt.axis("off")
      plt.imshow(resized_image)
      plt.show()

def gad(image_path):

    device = 'cpu'
    print('Running on device: {}'.format(device))
    model= InceptionResnetV1(classify=False,pretrained='vggface2').to(device)
    #model.load_state_dict(torch.load('modelsepoch_900.pth'))  
    #print(dir(model))
  
    target_layers = [model.repeat_1]
    rgb_img = cv2.imread(image_path, 1)[:, :, ::-1]
    rgb_img = np.float32(rgb_img) / 255
    input_tensor = preprocess_image(rgb_img,
                                    mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])

    targets = None
    cam_algorithm = LayerCAM
    with cam_algorithm(model=model,
                       target_layers=target_layers,
                       use_cuda=False) as cam:
        cam.batch_size = 32
        grayscale_cam = cam(input_tensor=input_tensor,targets=targets,aug_smooth=False,eigen_smooth=False)
        grayscale_cam = grayscale_cam[0, :]
        cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
        cam_image = cv2.cvtColor(cam_image, cv2.COLOR_RGB2BGR)
    gb_model = GuidedBackpropReLUModel(model=model, use_cuda=False)
    gb = gb_model(input_tensor, target_category=None)
    cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
    cam_gb = deprocess_image(cam_mask * gb)
    gb = deprocess_image(gb)

    cv2.imwrite(f'cam.jpg', cam_image)
    cv2.imwrite(f'gb.jpg', gb)
    cv2.imwrite(f'cam_gb.jpg', cam_gb)
    show('cam.jpg')
for usr in os.listdir('new-cluster_160'):
    for image in glob.glob(os.path.join('new-cluster_160', usr)+'/*.png'):
        gad(image)
        break
